# Using CROSS APPLY to optimize joins on BETWEEN conditions
https://visweshkk.blogspot.com/2014/06/using-cross-apply-to-optimize-joins-on.html

The slow query 

The following tables store one-minute commercials for every minute for one year, and customer calls, one call per minute, for the same year. The scripts that populate tables with test data are provided at the end of this post. Here are the tables:

Lets create the tables

In [25]:
if exists (select * from sys.objects where name = 'Commercials' and type = 'u')
    drop table dbo.Commercials

CREATE TABLE dbo.Commercials
(
    StartedAt DATETIME NOT NULL
        CONSTRAINT PK_Commercials PRIMARY KEY
  , EndedAt DATETIME NOT NULL
  , CommercialName VARCHAR(30) NOT NULL
);
GO

if exists (select * from sys.objects where name = 'Calls' and type = 'u')
    drop table dbo.Calls

CREATE TABLE dbo.Calls
(
    CallID INT
        CONSTRAINT PK_Calls NOT NULL PRIMARY KEY
  , AirTime DATETIME NOT NULL
  , SomeInfo CHAR(300)
);
GO

CREATE UNIQUE INDEX Calls_AirTime
ON dbo.Calls (AirTime)
INCLUDE (SomeInfo);
GO

Commands completed successfully.

Total execution time: 00:00:00.0110448

Commands completed successfully.

Total execution time: 00:00:00.0122618

Commands completed successfully.

Total execution time: 00:00:00.0276397

## Setting up tables and test data

In [26]:

if exists (select * from sys.objects where name = 'Numbers' and type = 'u')
    drop table dbo.Numbers


CREATE TABLE dbo.Numbers
(
    n INT NOT NULL PRIMARY KEY
);
GO

DECLARE @i INT;
SET @i = 1;
INSERT INTO dbo.Numbers
(
    n
)
SELECT 1;
WHILE @i < 1024000
BEGIN
    INSERT INTO dbo.Numbers
    (
        n
    )
    SELECT n + @i
    FROM dbo.Numbers;
    SET @i = @i * 2;
END;
GO
INSERT INTO dbo.Commercials
(
    StartedAt
  , EndedAt
  , CommercialName
)
SELECT DATEADD(MINUTE, n - 1, '20080101')
     , DATEADD(MINUTE, n, '20080101')
     , 'Show #' + CAST(n AS VARCHAR(6))
FROM dbo.Numbers
WHERE n <= 24 * 365 * 60;
GO
INSERT INTO dbo.Calls
(
    CallID
  , AirTime
  , SomeInfo
)
SELECT n
     , DATEADD(MINUTE, n - 1, '20080101')
     , 'Call during Commercial #' + CAST(n AS VARCHAR(6))
FROM dbo.Numbers
WHERE n <= 24 * 365 * 60;
GO

Commands completed successfully.

Total execution time: 00:00:00.0094968

(1 row affected)

(1 row affected)

(2 rows affected)

...

Total execution time: 00:00:04.0779781

(525600 rows affected)

Total execution time: 00:00:01.4483690

(525600 rows affected)

Total execution time: 00:00:05.2582609

## Every commercial in my table lasts for at most one minute, and they do not overlap. I can easily enforce both conditions with constraints </br>
## (Storing intervals of time with no overlaps" ), which are omitted in this post just to keep it simple.
##  The following query retrieves only 181 rows, and it runs **very slowly**: 

In [27]:
set STATISTICS TIME on
set STATISTICS IO on

SELECT s.StartedAt, s.EndedAt, c.AirTime
FROM dbo.Commercials s INNER JOIN dbo.Calls c
    ON c.AirTime >= s.StartedAt AND c.AirTime < s.EndedAt
WHERE c.AirTime BETWEEN '20080701' AND '20080701 03:00'    

set STATISTICS TIME OFF
set STATISTICS IO OFF


SQL Server Execution Times:
 CPU time = 0 ms, elapsed time = 0 ms.

(181 rows affected)

Table 'Calls'. Scan count 525600, logical reads 1696586, physical reads 0, read-ahead reads 7, lob logical reads 0, lob physical reads 0, lob read-ahead reads 0.

Table 'Commercials'. Scan count 9, logical reads 2805, physical reads 0, read-ahead reads 0, lob logical reads 0, lob physical reads 0, lob read-ahead reads 0.

Table 'Worktable'. Scan count 0, logical reads 0, physical reads 0, read-ahead reads 0, lob logical reads 0, lob physical reads 0, lob read-ahead reads 0.

SQL Server Execution Times:
 CPU time = 2547 ms, elapsed time = 375 ms.

Total execution time: 00:00:00.5523239

StartedAt,EndedAt,AirTime
2008-07-01 00:00:00.000,2008-07-01 00:01:00.000,2008-07-01 00:00:00.000
2008-07-01 00:01:00.000,2008-07-01 00:02:00.000,2008-07-01 00:01:00.000
2008-07-01 00:02:00.000,2008-07-01 00:03:00.000,2008-07-01 00:02:00.000
2008-07-01 00:03:00.000,2008-07-01 00:04:00.000,2008-07-01 00:03:00.000
2008-07-01 00:04:00.000,2008-07-01 00:05:00.000,2008-07-01 00:04:00.000
2008-07-01 00:05:00.000,2008-07-01 00:06:00.000,2008-07-01 00:05:00.000
2008-07-01 00:06:00.000,2008-07-01 00:07:00.000,2008-07-01 00:06:00.000
2008-07-01 00:07:00.000,2008-07-01 00:08:00.000,2008-07-01 00:07:00.000
2008-07-01 00:08:00.000,2008-07-01 00:09:00.000,2008-07-01 00:08:00.000
2008-07-01 00:09:00.000,2008-07-01 00:10:00.000,2008-07-01 00:09:00.000


 SQL Server Execution Times:
   CPU time = 0 ms,  elapsed time = 1 ms.

**(181 rows affected)**

**Table 'Calls'. Scan count 525600, logical reads 1706675, physical reads 2620, read-ahead reads 184, lob logical reads 0, lob physical reads 0, lob read-ahead reads 0.**

Table 'Commercials'. Scan count 9, logical reads 2793, physical reads 0, read-ahead reads 2487, lob logical reads 0, lob physical reads 0, lob read-ahead reads 0.

Table 'Worktable'. Scan count 0, logical reads 0, physical reads 0, read-ahead reads 0, lob logical reads 0, lob physical reads 0, lob read-ahead reads 0.


**SQL Server Execution Times:**

CPU time = 1142 ms,  elapsed time = 6352 ms.

Total execution time: 00:00:16.400

## Why is it so slow? I haven't mastered the fine art of adding images to my posts yet, so I have to explain verbally. For every call the DB engine scans all the commercials which begin before the time of the call, which is expensive. The reason is simple: the optimizer does not know that the commercials are short, and that the commercials do not overlap, so it must scan all the potential matches, which are all the commercials which begin before the time of the call.

In [28]:
set STATISTICS TIME on
set STATISTICS IO on


SELECT s.StartedAt, s.EndedAt, c.AirTime FROM dbo.Calls c CROSS APPLY( 
  SELECT TOP 1 s.StartedAt, s.EndedAt FROM dbo.Commercials s  
  WHERE c.AirTime >= s.StartedAt AND c.AirTime < s.EndedAt 
  ORDER BY s.StartedAt DESC) AS s WHERE c.AirTime BETWEEN '20080701' AND '20080701 03:00'

set STATISTICS TIME OFF
set STATISTICS IO OFF


SQL Server Execution Times:
 CPU time = 0 ms, elapsed time = 0 ms.

(181 rows affected)

Table 'Commercials'. Scan count 181, logical reads 603, physical reads 0, read-ahead reads 0, lob logical reads 0, lob physical reads 0, lob read-ahead reads 0.

Table 'Calls'. Scan count 1, logical reads 11, physical reads 0, read-ahead reads 0, lob logical reads 0, lob physical reads 0, lob read-ahead reads 0.

SQL Server Execution Times:
 CPU time = 0 ms, elapsed time = 1 ms.

Total execution time: 00:00:00.0145849

StartedAt,EndedAt,AirTime
2008-07-01 00:00:00.000,2008-07-01 00:01:00.000,2008-07-01 00:00:00.000
2008-07-01 00:01:00.000,2008-07-01 00:02:00.000,2008-07-01 00:01:00.000
2008-07-01 00:02:00.000,2008-07-01 00:03:00.000,2008-07-01 00:02:00.000
2008-07-01 00:03:00.000,2008-07-01 00:04:00.000,2008-07-01 00:03:00.000
2008-07-01 00:04:00.000,2008-07-01 00:05:00.000,2008-07-01 00:04:00.000
2008-07-01 00:05:00.000,2008-07-01 00:06:00.000,2008-07-01 00:05:00.000
2008-07-01 00:06:00.000,2008-07-01 00:07:00.000,2008-07-01 00:06:00.000
2008-07-01 00:07:00.000,2008-07-01 00:08:00.000,2008-07-01 00:07:00.000
2008-07-01 00:08:00.000,2008-07-01 00:09:00.000,2008-07-01 00:08:00.000
2008-07-01 00:09:00.000,2008-07-01 00:10:00.000,2008-07-01 00:09:00.000


## Using another range condition to tell the optimizer that commercials are short. 

 Because commercials are short, there is no need to scan the commercials that start more than maximum commercial's length before the call. Again, translating this information into plain SQL is quite easy too, and again the query runs much faster, even faster than the previous one:


SQL Server Execution Times: CPU time = 0 ms, elapsed time = 0 ms.

**(181 rows affected)**
Table 'Commercials'. Scan count 181, logical reads 603, physical reads 0, read-ahead reads 0, lob logical reads 0, lob physical reads 0, lob read-ahead reads 0.

**Table 'Calls'. Scan count 1, logical reads 11, physical reads 0, read-ahead reads 0, lob logical reads 0, lob physical reads 0, lob read-ahead reads 0.**

SQL Server Execution Times: CPU time = 0 ms, elapsed time = 0 ms.

Total execution time: 00:00:00.0112655

In [29]:
    SELECT s.StartedAt, s.EndedAt, c.AirTime
    FROM dbo.Commercials s INNER JOIN dbo.Calls c
        ON c.AirTime >= s.StartedAt AND c.AirTime < s.EndedAt
    WHERE c.AirTime BETWEEN '20080701' AND '20080701 03:00'
EXCEPT
    SELECT s.StartedAt, s.EndedAt, c.AirTime
    FROM dbo.Calls c CROSS APPLY( 
  SELECT TOP 1
            s.StartedAt, s.EndedAt
        FROM dbo.Commercials s
        WHERE c.AirTime >= s.StartedAt AND c.AirTime < s.EndedAt
        ORDER BY s.StartedAt DESC) AS s
    WHERE c.AirTime BETWEEN '20080701' AND '20080701 03:00'

(0 rows affected)

Total execution time: 00:00:00.3660962

StartedAt,EndedAt,AirTime
